Load and Inspect
Checklist:
1. Identify the target column (e.g. salary_in_usd).
2. Note categorical vs numeric columns.
3. Check df.shape — how big is it?
4. Watch for object dtype columns that should be numeric.

In [23]:
import pandas as pd

df = pd.read_csv('../data/raw/eda_data.csv')  # adjust filename
df.head()
df.info()
df.describe()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,...,age,python_yn,R_yn,spark,aws,excel,job_simp,seniority,desc_len,num_comp
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,...,47,1,0,0,0,1,data scientist,na,2536,0
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,...,36,1,0,0,0,0,data scientist,na,4783,0
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,...,10,1,0,1,0,1,data scientist,na,3461,0
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,...,55,1,0,0,0,0,data scientist,na,3883,3
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,...,22,1,0,0,0,1,data scientist,na,2728,3


🔍 1. Understand what we have

Target variable:

avg_salary → numeric, already extracted (perfect for modeling).

Useful predictors:

Category	Columns
Job Info	Job Title, job_simp, seniority
Company Info	Company Name, Size, Founded, Type of ownership, Industry, Sector, Revenue, num_comp
Location	Location, job_state, same_state
Company Rating	Rating, age
Skills	python_yn, R_yn, spark, aws, excel
Others	desc_len, hourly, employer_provided

Columns to drop:

Unnamed: 0 (index)

Salary Estimate, Job Description (raw text)

Competitors, company_txt (duplicate of company name)

Possibly Company Name (too granular, not useful for model)
2. Cleaning and Feature Engineering Steps
(a) Drop irrelevant columns

In [25]:
df.drop(columns=[
    'Unnamed: 0', 'Salary Estimate', 'Job Description',
    'Competitors', 'company_txt', 'Company Name'
], inplace=True)

#Remove exact duplicates
df.drop_duplicates(inplace=True)

(b) Fix categorical formats
Standardize text columns to lowercase.
Replace “-1” or “unknown” with NaN.

In [27]:
import numpy as np
df.replace(-1, np.nan, inplace=True)
df = df.map(lambda s: s.lower().strip() if isinstance(s, str) else s)

(c) Simplify “Size” column
Convert size ranges to numeric midpoint:

In [30]:
def size_to_num(x):
    if 'to' in x:
        low, high = x.split('to')
        return (int(low.strip()) + int(high.split()[0])) // 2
    elif '10000+' in x:
        return 10000
    return np.nan

df.columns = (
    df.columns.str.strip()
              .str.lower())
df['company_size'] = df['size'].apply(size_to_num)
df.drop('size', axis=1, inplace=True)

In [ ]:
(d) Fix Founded and compute company age

In [31]:
df['company_age'] = 2025 - df['founded']
df.drop('founded', axis=1, inplace=True)

In [ ]:
(e) Simplify revenue to numeric range

In [32]:
def rev_to_num(x):
    if 'billion' in x:
        nums = [float(s.replace('$','').replace('billion','')) for s in x.split('to') if '$' in s]
        return np.mean(nums)*1000  # convert billion to million
    elif 'million' in x:
        nums = [float(s.replace('$','').replace('million','')) for s in x.split('to') if '$' in s]
        return np.mean(nums)
    else:
        return np.nan

df['revenue_million'] = df['revenue'].apply(rev_to_num)
df.drop('revenue', axis=1, inplace=True)


ValueError: could not convert string to float: ' 100  (usd)'

In [ ]:
(f) Encode binary features

They are already encoded as 0/1:

python_yn, R_yn, spark, aws, excel, same_state, hourly, employer_provided

No change needed.

In [ ]:
🧮 3. Scale numerical columns

In [ ]:
from sklearn.preprocessing import StandardScaler
num_cols = ['rating', 'age', 'company_age', 'company_size', 'revenue_million',
            'desc_len', 'num_comp', 'min_salary', 'max_salary']
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

In [ ]:
💾 4. Save the cleaned dataset

In [ ]:
df.to_csv('data/processed/cleaned_salary_data.csv', index=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(df['salary_in_usd'], bins=30)
plt.title("Salary Distribution")

sns.boxplot(x='experience_level', y='salary_in_usd', data=df)
plt.title("Salary by Experience Level")

sns.barplot(x='company_size', y='salary_in_usd', data=df)
plt.title("Salary by Company Size")
